In [1]:
input={
1:"count = 0",
2:"Result = 0",
3:"If count > 20 GOTO 8",
4:"count=count + 1",
5:"increment = 2 * count",
6:"result = result +increment",
7:"GOTO 3",
8:"end"
}

In [3]:
def find_leaders(input):
  leader_keys=[]
  for key,value  in input.items():
    if key==1:
      leader_keys.append(key)
    if 'If' in value:
      leader_keys.append(key)
      leader_keys.append(key+1)
    if 'GOTO' in value:
      leader_keys.append(int(value[-1]))
  return list(set(leader_keys))

In [4]:
leader_keys=sorted(find_leaders(input))
leader_keys

[1, 3, 4, 8]

In [5]:


def basic_block(leader_keys):
  basic_block_range=[]
  for i in range(len(leader_keys)-1):
    low=leader_keys[i]
    high=leader_keys[(i+1)]-1
    basic_block_range.append([low,high])
  basic_block_range.append([leader_keys[-1],leader_keys[-1]])
  return basic_block_range


In [6]:


bb_range=basic_block(leader_keys)
bb_range

[[1, 2], [3, 3], [4, 7], [8, 8]]

In [8]:

def create_blocks(bb_range):
  blocks=[]
  for low,high in bb_range:
    block=[]
    while(True):
      if(low>high):
        blocks.append(block)
        break
      block.append([low,input[low]])
      low+=1
  return blocks

In [9]:
blocks=create_blocks(bb_range)
for block in blocks:
    print(block)

[[1, 'count = 0'], [2, 'Result = 0']]
[[3, 'If count > 20 GOTO 8']]
[[4, 'count=count + 1'], [5, 'increment = 2 * count'], [6, 'result = result +increment'], [7, 'GOTO 3']]
[[8, 'end']]


In [11]:

def pfg(blocks):
  admatrix=[[0,0,0,0],
          [0,0,0,0],
          [0,0,0,0],
          [0,0,0,0]]
  for i in range(len(blocks)-1):
    block = blocks[i]
    for statement in block:
      # admatrix[i][i]=1
      if "If" in statement[-1]:
        for j in range(len(bb_range)):
           if int(statement[-1][-1])>=bb_range[j][0] and int(statement[-1][-1])<=bb_range[j][1]:
             admatrix[i][j]=1   
        admatrix[i][i+1]=1   
      else:
        if "GOTO" in statement[-1]:
           for j in range(len(bb_range)):
             if int(statement[-1][-1])>=bb_range[j][0] and int(statement[-1][-1])<=bb_range[j][1]:
               admatrix[i][j]=1
      if "GOTO" not in block[-1][-1]:
        admatrix[i][i+1]=1

  # print(admatrix)
  return list(admatrix)

In [12]:
admatrix=pfg(blocks)
admatrix

[[0, 1, 0, 0], [0, 0, 1, 1], [0, 1, 0, 0], [0, 0, 0, 0]]

In [13]:
def find_path(admatrix):
  paths=[]
  for i in range(len(admatrix)):
      paths.append([i,i])
      for j in range(len(admatrix)):
        if admatrix[i][j]==1:
          paths.append([i,j])
  return paths

In [15]:
paths=find_path(admatrix)
paths

[[0, 0], [0, 1], [1, 1], [1, 2], [1, 3], [2, 2], [2, 1], [3, 3]]

In [14]:
def find_dom(paths):
  doms=[]
  for i in range(len(admatrix)):
    dom=[]
    for j in paths:
      if i==j[-1] and i not in dom:
        dom.append(j[0])
    doms.append(dom)  
  # print(doms)
  return doms

In [16]:
doms=find_dom(paths)
doms

[[0], [0, 1], [1, 2], [1, 3]]

In [17]:
def find_back_edge(doms):
  for i in range(len(admatrix)):
    for j in range(len(admatrix)):
      head=i
      if admatrix[i][j]==1:
        tail=j
        if head not in doms[j]:
          print('backedge between B',i+1,'and B',j+1)


In [ ]:
find_back_edge(doms)

backedge between B 3 and B 2
